In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import tensorflow as tf 

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
!pwd

/kaggle/working


In [3]:
train_dir = "../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
test_dir = "../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"

In [4]:
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                 batch_size=32
                                                                )
                                                                                
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False,
                                                                batch_size = 32
                                                               ) 

Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.


In [5]:
checkpoint_path = "disease"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, 
                                                         monitor="val_accuracy", 
                                                         save_best_only=True) 

In [6]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential


data_augmentation = Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2), 
  preprocessing.RandomHeight(0.2), 
  preprocessing.RandomWidth(0.2), 
  preprocessing.RandomZoom(0.2), 
], name="data_augmentation")

In [7]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False


inputs = layers.Input(shape=(224, 224, 3), name="input_layer") 
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x) 
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

16705208/16705208 [==============================] - 0s 0us/step


In [8]:
print(tf.config.list_physical_devices('TPU'))

[]


In [9]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=["accuracy"])

In [10]:


history = model.fit(train_data,
                    epochs=5,
                    validation_data=test_data,
                    validation_steps=int(0.15 * len(test_data)),
                    batch_size=32,  
                    callbacks=[checkpoint_callback]
                   )


Epoch 1/5
2197/2197 [==============================] - 454s 198ms/step - loss: 0.4872 - accuracy: 0.8919 - val_loss: 0.2105 - val_accuracy: 0.9444
Epoch 2/5
2197/2197 [==============================] - 254s 115ms/step - loss: 0.1748 - accuracy: 0.9517 - val_loss: 0.1241 - val_accuracy: 0.9680
Epoch 3/5
2197/2197 [==============================] - 209s 95ms/step - loss: 0.1312 - accuracy: 0.9628 - val_loss: 0.1236 - val_accuracy: 0.9634
Epoch 4/5
2197/2197 [==============================] - 186s 85ms/step - loss: 0.1076 - accuracy: 0.9683 - val_loss: 0.1087 - val_accuracy: 0.9684
Epoch 5/5
2197/2197 [==============================] - 174s 79ms/step - loss: 0.0940 - accuracy: 0.9721 - val_loss: 0.0789 - val_accuracy: 0.9764


In [11]:
results_feature_extraction_model = model.evaluate(test_data)
results_feature_extraction_model

550/550 [==============================] - 47s 86ms/step - loss: 0.1265 - accuracy: 0.9586


[0.12649846076965332, 0.9586273431777954]

In [12]:
model.save_weights("disease_model_weights.h5")


In [14]:
model.save('disease.h5')

TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [15]:
tf.__version__

'2.12.0'

In [19]:
class_names = train_data.class_names
class_names


['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_